In [44]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


1. Для нашего пайплайна (Case1) поэкспериментировать с разными моделями: 1 - бустинг, 2 - логистическая регрессия (не забудьте здесь добавить в cont_transformer стандартизацию - нормирование вещественных признаков)
2. Отобрать лучшую модель по метрикам (кстати, какая по вашему мнению здесь наиболее подходящая DS-метрика)
3. Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2 (1 доллар на привлечение, 2 доллара - с каждого правильно классифицированного (True Positive) удержанного). (подсказка) нужно посчитать FP/TP/FN/TN для выбранного оптимального порога вероятности и посчитать выручку и траты. 
4. (опционально) Провести подбор гиперпараметров лучшей модели по итогам 2-3
5. (опционально) Еще раз провести оценку экономической эффективности

__Задание 1__ Для нашего пайплайна (Case1) поэкспериментировать с разными моделями: 1 - бустинг, 2 - логистическая регрессия (не забудьте здесь добавить в cont_transformer стандартизацию - нормирование вещественных признаков)
<br>
<br>

__Решение:__ загружаю данные и переписываю класс обработки признаков по-своему.

In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
import numpy as np
import pandas as pd


from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from catboost import CatBoostClassifier 
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier

from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, KFold 

from sklearn.metrics import f1_score, roc_auc_score, log_loss, precision_recall_curve

%matplotlib inline

pd.set_option("display.max_rows", 6)

In [47]:
with open('/content/drive/MyDrive/ГБ/выборки для исследований/churn_data.csv', "rb") as f:
    df = pd.read_csv(f)
    
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [48]:
y = df['Exited']
X = df.drop(columns='Exited')

In [49]:
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [50]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
dtypes: float64(2), int64(8), object(3)
memory usage: 1015.8+ KB


'Tenure', 'HasCrCard', 'IsActiveMember' нужно перевести в тип данных объект

In [51]:
X = X.astype({'Tenure': np.object, 'HasCrCard': np.object, 'IsActiveMember': np.object})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.


__Не понял зачем в нотбуке с урока приведен код формирования единого пайпайна обработки признаков церез цикл, если можно сделать проще как у меня здесь с единым классов ColumnSelector для кат. и числовых признаков. Или я что-то делаю не так?__


__Так же не понял почему в описаниях к заданиям в ходе занятий указывается на использование стандартизации, при том что считается что в моделях классификации нужно использовать нормализвацию (MinMaxSaler)__

Мой вариант:

In [52]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]

class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

In [53]:
num_transfomer =  Pipeline([
            ('selector', ColumnSelector(key=continuous_columns)),
            ('standard', MinMaxScaler())
        ])

cat_transformer = Pipeline([
            ('selector', ColumnSelector(key=categorical_columns)),
            ('ohe', OHEEncoder(key=categorical_columns))
        ])
  
feats = FeatureUnion([('transfomer_num', num_transfomer),
                      ('cat_transformer', cat_transformer)
                      ])

#feature_processing = Pipeline([('sel', feats)])


In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.3)

In [55]:
# Пропишем алгоритм подсчета метрик для наших будущих моделей

results = pd.DataFrame(columns=['model', 'thresh', 'F-Score', 'Precision', 'Recall', 'ROC AUC'])

def compute_result(model_name, preds_proba, results):
    precision, recall, thresholds = precision_recall_curve(y_test, preds_proba)
    fscore = (2 * precision * recall) / (precision + recall)
    auc = roc_auc_score(y_test, preds_proba)
    ix = np.argmax(fscore)

    results = results.append({
      'model': model_name,
      'thresh': thresholds[ix],
      'F-Score': fscore[ix],
      'Precision': precision[ix],
      'Recall': recall[ix],
      'ROC AUC': auc
      }, ignore_index=True)
    
    return results



Все готово для тестирования разных моделей и поиска их лучших гиперпараметров.

Поиск лучшей модели буду проводить отдельно для каждой модели, а не в едином списке параметров с передаче новых моделей МО и их гиперпараметров, т.к. во втором случае придется слишком долго ждать результат. Хочу быстрее находить итог для каждой модели.

<br>

__С точки зрения корректности отладки алгоритмов использование пайплайна в данном случае яв-ся совсем не оптимальным, т.к. при поиске лучшей модели он запускается каждый раз заново в новой модели. Оптимальным было бы сразу трансформировать X и уже его разбивать на трейн и тест. Просто мысли в слух__

In [56]:
# Начнем с гр. бустинга
# %%time


# gb_model = Pipeline([
#     ('preprocessing', feats),
#     ('classifier', GradientBoostingClassifier(random_state=1))
# ])


# params = [{#'classifier__criterion': ['friedman_mse', 'squared_error', 'mse'], # долго все перебирать
#            'classifier__max_depth': [5,6], # range(4,7)
#            'classifier__n_estimators':  [50, 100, 300, 500], # 750, 1000
#            'classifier__learning_rate': [0.01, 0.1, 1], #, 10, 100
#            'classifier__min_samples_leaf': [1, 2, 4]
#            }]

# grid_gb = GridSearchCV(gb_model, params, cv=5, n_jobs=-1)
# grid_gb.fit(X_train, y_train)
# grid_gb.best_params_

# --------------------------
# CPU times: user 24.9 s, sys: 1.39 s, total: 26.3 s
# Wall time: 20min 56s
# {'classifier__learning_rate': 0.01,
#  'classifier__max_depth': 5,
#  'classifier__min_samples_leaf': 4,
#  'classifier__n_estimators': 500}


In [57]:
# Запишем пайплайн гб с лучшими гиперпараметрами и его метрики
gb_model = Pipeline([
    ('preprocessing', feats),
    ('classifier', GradientBoostingClassifier(random_state=1, learning_rate=0.01, max_depth=5, min_samples_leaf=4, n_estimators=500))
])

gb_model.fit(X_train, y_train)

pred_gb = gb_model.predict_proba(X_test)[:, 1]

results = compute_result('GradientBoosting', pred_gb, results)


Теперь потестим логистическую регрессию:

In [58]:
# %%time

# model_lg = Pipeline([
#     ('preprocessing', feats),
#     ('classifier', LogisticRegression(random_state=1))
# ])

# params_grid = [{'classifier__C': [0.01, 0.1, 1, 10, 100],
#                'classifier__max_iter': [50, 100, 200, 300, 500],
#                'classifier__solver': ['lbfgs', 'liblinear']}
#                ]
                

# grid_lg = GridSearchCV(model_lg, params_grid, cv=5, n_jobs=-1)
# grid_lg.fit(X_train, y_train)
# grid_lg.best_params_

# ---------------------
# CPU times: user 5.53 s, sys: 297 ms, total: 5.83 s
# Wall time: 23 s
# {'classifier__C': 0.1,
#  'classifier__max_iter': 50,
#  'classifier__solver': 'lbfgs'}

In [59]:
# Запишем пайплайн лог. регрессии с лучшими гиперпараметрами и его метрики

model_lg = Pipeline([
    ('preprocessing', feats),
    ('classifier', LogisticRegression(C=0.1, max_iter=50, solver='lbfgs',random_state=1))
    ])

model_lg.fit(X_train, y_train)

pred_lg = model_lg.predict_proba(X_test)[:, 1]

results = compute_result('LogisticRegression', pred_lg, results)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


Лог. регрессия на что-то ругнулась...

Перейдем к модели на основе метода опорных векторов. Если в данных какой-то линейный косяк, который встретила лог. регрессия, то и на svc мы это увидим

In [60]:
# %%time

# model_svc = Pipeline([
#     ('preprocessing', feats),
#     ('classifier', SVC(random_state=1))
# ])

# params_grid = [{'classifier__gamma': [0.001, 0.01, 0.1, 1, 10, 100],
#                 'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]
#                 }]
                

# grid_svc = GridSearchCV(model_svc, params_grid, cv=5, n_jobs=-1)
# grid_svc.fit(X_train, y_train)
# grid_svc.best_params_


# #--------------------
# CPU times: user 8.77 s, sys: 647 ms, total: 9.41 s
# Wall time: 5min 12s
# {'classifier__C': 100, 'classifier__gamma': 0.1}

In [61]:
# Запишем пайплайн svc с лучшими гиперпараметрами и его метрики

model_svc = Pipeline([
    ('preprocessing', feats),
    ('classifier', SVC(random_state=1, C=100, gamma=0.1))
    ])

model_svc.fit(X_train, y_train)

pred_svc = model_svc.decision_function(X_test)

results = compute_result('SVC', pred_svc, results)

Перейдем к модели экстремальных деревьев:

In [62]:
# %%time

# from sklearn.ensemble import ExtraTreesClassifier

# model_extr = Pipeline([
#     ('preprocessing', feats),
#     ('classifier', ExtraTreesClassifier(random_state=1))
#     ])



# params_grid = [{'classifier__n_estimators': [30, 50, 100, 150, 200],
#                 'classifier__criterion': ['gini', 'entropy'],
#                 'classifier__min_samples_leaf': [1,2,4,8]
#                 }]
                

# grid_extr = GridSearchCV(model_extr, params_grid, cv=5, n_jobs=-1)
# grid_extr.fit(X_train, y_train)
# grid_extr.best_params_


# # ---------------------
# CPU times: user 5.67 s, sys: 286 ms, total: 5.96 s
# Wall time: 1min 49s
# {'classifier__criterion': 'entropy',
#  'classifier__min_samples_leaf': 4,
#  'classifier__n_estimators': 100}

In [63]:
from sklearn.ensemble import ExtraTreesClassifier

model_extr = Pipeline([
    ('preprocessing', feats),
    ('classifier', ExtraTreesClassifier(criterion='entropy', min_samples_leaf=4, random_state=1)) # n_estimators по умолчанию итак 100
    ])

model_extr.fit(X_train, y_train)

preds = model_extr.predict_proba(X_test)[:, 1]
results = compute_result('ExtraTrees', preds, results)

__Задание 2__ Отобрать лучшую модель по метрикам (кстати, какая по вашему мнению здесь наиболее подходящая DS-метрика)

__Решение:__ выведем таблицу сравнения метрик

In [64]:
results


,model,thresh,F-Score,Precision,Recall,ROC AUC
0,GradientBoosting,0.385376,0.646483,0.731855,0.578947,0.870031
1,LogisticRegression,0.842417,NaN,0.000000,0.000000,0.756649
2,SVC,-0.423594,0.598383,0.685185,0.531100,0.818572
3,ExtraTrees,0.323107,0.609477,0.624791,0.594896,0.842716


Видим что с логистической регрессией что-то нет то, при этом svc отработал корректно.

Лучше всего отрабатывает Градиентный бустинг, т.к. имеем самые высокие fscore и roc auc

__Задание 3__ <br>
Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2 (1 доллар на привлечение, 2 доллара - с каждого правильно классифицированного (True Positive) удержанного). (подсказка) нужно посчитать FP/TP/FN/TN для выбранного оптимального порога вероятности и посчитать выручку и траты. 

__Решение:__ 
<br>

Для начала нужно сформировать предсказания классов у нашей модели гр. бустинга (ранее вызывали только predict_proba).<br>
Модель должна предсказывать класс, исходя из найденного порога 0.385376.

Я не понял как заставить модель предсказывать на основе найденного порога. Так, у самой модели гр. бустинга нет параметра порог, у метода predict() тоже отсутствует параметр порог.

Поэтому стало интересно сделать сравнительную таблицу, куда передать preds_proba, y_pred из метода .predict(), preds by tresholds где я вручную проставляю класс в зависимости от значение preds_proba к порогу.



In [65]:
cf = pd.DataFrame(columns=['y_test', 'pred_proba', 'y_preds', 'preds by tresholds'])
cf.y_test = y_test
cf.pred_proba = pred_gb
cf.y_preds = gb_model.predict(X_test)
cf['preds by tresholds'] = np.where((cf.pred_proba > 0.385376), 1, 0)

# Сделаем сравнение того, что предсказыват метод .predict с тем, что мы рассчитали вручную, ориентируясь на значение порога
cf['check_preds'] = cf.y_preds == cf['preds by tresholds']

# Посмотрим в скольких случаях наблюдаем разницу с определением класса
cf.check_preds.value_counts()

True     2890
False     110
Name: check_preds, dtype: int64

In [66]:
# посмотрим детально несколько строк, где не совпало

cf.loc[cf.check_preds == False, :]

,y_test,pred_proba,y_preds,preds by tresholds,check_preds
6426,0,0.440509,0,1,False
9781,1,0.405889,0,1,False
6849,0,0.402521,0,1,False
...,...,...,...,...,...
3165,0,0.408768,0,1,False
1051,0,0.393179,0,1,False
8780,1,0.459598,0,1,False


Сравним конф. матрицы обоих алгоритмов:

In [67]:
pd.crosstab(y_test, cf.y_preds)

y_preds,0,1
Exited,,
0,2290,83
1,324,303


In [68]:
pd.crosstab(y_test, cf['preds by tresholds'])

preds by tresholds,0,1
Exited,,
0,2240,133
1,264,363


Видим что предсказания, которые мы рассчитали вручную на основе порога, лучше предсказывают TN, следовательно мы на 60 наблюдений можем сформировать список клиентов больше для будущего взаимодействия с ними

Из условия задачи следует, что:<br>
-  затраты на привлечение 1 клиента = 1 доллар,
-  привлеченный сразу становится TP,
-  все ТP приносят доход 2 доллара (и привлеченный, и удержанный) 

In [88]:
res = pd.DataFrame(columns=['type', '50', '100', '200', '400'])


In [89]:
TP = 2240
num_people_to_experiment = [50, 100, 200, 400]


# удержать клиентов
res = res.append({
  'type': 'расходы на привлечение',
  '50': 50 * 1,
  '100': 100 * 1,
  '200': 200 * 1,
  '400': 400 * 1
  }, ignore_index=True)

# чистая прибыль с новых  клиентов:
res = res.append({
  'type': 'доход с новых  клиентов',
  '50': 50 * 2 - 50,
  '100': 100 * 2 - 100,
  '200': 200 * 2 - 200,
  '400': 400 * 2 - 400
  }, ignore_index=True)

# привлечь новых клиентов:
res = res.append({
  'type': 'доход при удержании',
  '50': 50 * 2,
  '100': 100 * 2,
  '200': 200 * 2,
  '400': 400 * 2
  }, ignore_index=True)



In [90]:
res

,type,50,100,200,400
0,расходы на привлечение,50,100,200,400
1,доход с новых клиентов,50,100,200,400
2,доход при удержании,100,200,400,800


Отсюда видим что, привлекая клиентов, мы не работаем в минус, а работая над удержанием, мы получаем стабильный доход вдвое выше от дохода от нового клиента.

Важно знать расходы на удержание для выбора корректной стратегии